In [37]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 

In [2]:
if(False):
    i=0
    for caption in webvtt.read('/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why_Can\'t_You_Fly%3F_deu.vtt'):
        print(caption)
        print(i)
        print('----------')
        print(caption.start) # start timestamp in text format
        print(caption.end) # end timestamp in text format
        print(caption.text) # caption text
        i=i+1

In [32]:
rootdir = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/"
original_file_backup = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/original_File_Backup"

In [2]:
#1)
"""
converting all audio to wav file with the following specification. 
    -WAVE files with 16-bit
    -16 kHz
    -mono 
"""
for Dir in os.listdir(rootdir):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            print("file already converted")
        else:
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir): 
                if(file.endswith(".wav")):
                    actual_filename = file[:-4]+"_C"
                    a = os.system('ffmpeg -i {} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(rootdir+"/"+Dir+"/"+subdir+"/"+file, rootdir+"/"+Dir+"/"+subdir, actual_filename))# (source,target,name_of_file)
                    if(a==256):
                        print(file+" : Did not get converted")
                        
#------------------------------------------------Backing up----------------------------------------------------------

for Dir in os.listdir(rootdir):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            #move the file
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
                if(file.endswith(".wav") and file.split('_')[-1]!="C.wav"):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
        if(length == 2):
            print(subdir+ " is being removed")
            #move complete folder the the backup folder
            shutil.move(rootdir+"/"+Dir+"/"+subdir, original_file_backup) # (source,target)
            

----------------------SUB-37---------------------
file already converted
----------------------SUB-49---------------------
Why Is Friday The 13th Unlucky_mixdown_ln_13th_DEU.wav : Did not get converted
----------------------SUB-55---------------------
file already converted
----------------------SUB-43---------------------
file already converted
----------------------SUB-31---------------------
STOP Your Bathroom Mirror from Fogging Up!_mixdown_hh_Stop_DEU.wav : Did not get converted
----------------------SUB-25---------------------
file already converted
----------------------SUB-19---------------------
file already converted
----------------------SUB-13---------------------
file already converted
----------------------SUB-6---------------------
file already converted
----------------------SUB-67---------------------
file already converted
----------------------SUB-79---------------------
file already converted
----------------------SUB-61---------------------
file already converted
-

In [58]:
# 2)
"""
creating chunks and saving in the same folder
"""
for Dir in os.listdir(rootdir):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("1. -----------------------"+subdir)
        df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".vtt")):                
                for caption in webvtt.read(rootdir+"/"+Dir+"/"+subdir+"/"+file):
                    df2 = pd.DataFrame([[caption.start,caption.end,caption.text]], columns=list('ABC'))
                    df = df.append(df2)
                df = df.rename(index=str, columns={"A": "start", "B": "end","C" : "text"})
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".wav") and file.split('_')[-1]=="C.wav"):                
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/"+subdir+"/"+file, "wav")
                for i in range(0, len(df)):
                    start_time = df['start'].iloc[i]
                    end_time = df['end'].iloc[i]
                    # -----------------------------START-TIME-----------------------------------
                    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
                    s_hours = int(s_hours)
                    s_minutes = int(s_minutes)
                    s_seconds = float(s_seconds)
                    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
                    START = s_millisecond
                    #-------------------------------END-TIME---------------------------------------
                    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
                    e_hours = int(e_hours)
                    e_minutes = int(e_minutes)
                    e_seconds = float(e_seconds)
                    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
                    END = e_miliseconds
                    #--------------------------------------------------------------------------
                    chunk = myaudio[START:END]
                    chunk_name = file.split('.')[-2]+"_"+str(i)+".wav"
                    if not os.path.exists(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name):
                        print("exporting", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name, format="wav")
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                    print("Backing up")
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
#----------------------------------------------txt-----------------------------------------------------------------            
        txt_name = ""        
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                txt_name = file.split(".")[-2]
        with open(rootdir+"/"+Dir+"/"+subdir+"/"+txt_name+"_C.trans.txt", "w") as txt_file:
            i=0
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):                
                if(file.endswith(".wav")):
                    wav_file_name = file.split(".")[-2]
                    print(df['text'].iloc[i])
                    txt_file.write(wav_file_name+" "+df['text'].iloc[i]+"\n")
                    i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
               
               
            

1. -----------------------SUB-67
Hallo und willkommen
bei Life Noggin. Es ist Zeit für ein
weiteres unserer gefragtesten Videos. Wir widerlegen weitere
populäre Mythen. Bist du aufgeregt?
Ich bin aufgeregt!
Legen wir los!
Uns allen wurde wahrscheinlich beigebracht,
dass unsere Zunge in verschiedene
Geschmackszonen “aufgeteilt” werden kann.
Also könnten wir auf einen bestimmten Bereich zeigen und sagen
“Da ist er! Das ist der Punkt der Kuchen so schmackhaft macht.”
Das ist eindeutig falsch.
Die fünf Geschmacksrichtungen können überall auf der Zunge wahrgenommen werden, besonders an den Seiten.
Das einzige was an diesem Mythos auch nur
entfernt stimmt, ist,
dass der hintere Teil unserer Zunge sehr sensibel auf bittere Geschmäcker reagiert.
Nächster Mythos! Glaub nicht den populären Mythos, dass Fledermäuse blind seien.
In Wirklichkeit können Fledermäuse sehr gut sehen.
Sie können auch mit ihrer Echoortung nachts Insekten jagen und sich zurechtzufinden.
Wenn du also das nächste mal
aus we

In [21]:
SAVE_DIR = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/"
dirName = "Why Can't You Fly_mixdown_ln_cantFly_DEU" # direct file name as directory name
dirPath = SAVE_DIR+"/"+dirName
#here you put a loop to read all the wav files
myaudio = AudioSegment.from_file("/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why Can't You Fly_mixdown_ln_cantFly_DEU.wav", "wav")
for i in range(0, len(df)):
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)    
    start_time = df['start'].iloc[i]
    end_time = df['end'].iloc[i]    
    # ------------------------------------------------------------------------
    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
    s_hours = int(s_hours)
    s_minutes = int(s_minutes)
    s_seconds = float(s_seconds)
    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
    START = s_millisecond
    #--------------------------------------------------------------------------
    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
    e_hours = int(e_hours)
    e_minutes = int(e_minutes)
    e_seconds = float(e_seconds)
    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
    END = e_miliseconds
    #--------------------------------------------------------------------------
    chunk = myaudio[START:END]    
    chunk_name =  str(i)+".wav" 
    file_destination = dirPath+"/"+chunk_name
    if not os.path.exists(file_destination):
        print("exporting", chunk_name)
        chunk.export(file_destination , format="wav")



exporting 0.wav
exporting 1.wav
exporting 2.wav
exporting 3.wav
exporting 4.wav
exporting 5.wav
exporting 6.wav
exporting 7.wav
exporting 8.wav
exporting 9.wav
exporting 10.wav
exporting 11.wav
exporting 12.wav
exporting 13.wav
exporting 14.wav
exporting 15.wav
exporting 16.wav
exporting 17.wav
exporting 18.wav
exporting 19.wav
exporting 20.wav
exporting 21.wav
exporting 22.wav
exporting 23.wav
